In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pyodbc
import warnings

warnings.filterwarnings("ignore")

## Users

In [10]:
cnx: pyodbc.Connection = pyodbc.connect(
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=DWH;"
    "Database=dwOper;"
    "Trusted_Connection=yes;")
query = f"""
DROP TABLE IF EXISTS #users
SELECT TOP 500 PartnerUserID
INTO #users
FROM VIEW_PlatformPartnerUsers_TotogamingAm
ORDER BY NEWID()

SELECT a.TOTOID,
    SUM(a.DepositCount) AS DepositCount,
    SUM(a.DepositAmount) AS DepositAmount,
    a.Age AS Age,
    a.RegDate,
    a.EndDate
FROM
(SELECT cu.PartnerUserID AS TOTOID,
    COUNT(p.PaymentID) DepositCount,
    SUM(p.Amount) DepositAmount,
    MAX(DATEDIFF(YEAR, cu.BirthDate, CAST(GETDATE() AS DATE))) AS Age,
    CAST(cu.RegistrationDate AS DATE) AS RegDate,
    CAST(DATEADD(MONTH, 6, CAST(cu.RegistrationDate AS DATE)) AS DATE) AS EndDate
FROM Payment p
INNER JOIN VIEW_PlatformPartnerUsers_TotogamingAm cu ON p.UserID = cu.UserID
LEFT JOIN C_PaymentSystem sp ON sp.PaymentSystemId = p.PaymentSystemID
WHERE CAST(p.modify_date AS DATE) <= CAST(DATEADD(MONTH, 6, CAST(cu.RegistrationDate AS DATE)) AS DATE)
    AND cu.PartnerUserID IN (SELECT PartnerUserID FROM #users)
	AND p.PaymentStatusID = 8
    AND p.PaymentTypeID = 2
    AND p.SourceID = 2 --Casino Source
    AND sp.PaymentSystemName NOT IN ('PokerTransfer','TRANSFER')
GROUP BY cu.PartnerUserID, cu.RegistrationDate

UNION ALL

SELECT cu.PartnerUserID AS TOTOID,
    COUNT(p.PaymentID) DepositCount,
    SUM(p.Amount) DepositAmount,
    MAX(DATEDIFF(YEAR, cu.BirthDate, CAST(GETDATE() AS DATE))) AS Age,
    CAST(cu.RegistrationDate AS DATE) AS RegDate,
    CAST(DATEADD(MONTH, 6, CAST(cu.RegistrationDate AS DATE)) AS DATE) AS EndDate
    FROM Payment p
INNER JOIN VIEW_sport_PartnerUser_TotogamingAm cu ON p.UserID = cu.UserID
LEFT JOIN C_PaymentSystem sp ON sp.PaymentSystemId = p.PaymentSystemID
WHERE CAST(p.modify_date AS DATE) <= CAST(DATEADD(MONTH, 6, CAST(cu.RegistrationDate AS DATE)) AS DATE)
	AND cu.PartnerUserID IN (SELECT PartnerUserID FROM #users)
    AND p.TransactionTypeID IN (62,65,70,73,87,88,102,103,75,71)
    AND p.Fund_StatusID IN (3,23,31,53,63,106)
    AND p.SourceID = 1  --Sport Source
    AND sp.PaymentSystemName NOT IN ('Casino Transfer','BONUS')
GROUP BY cu.PartnerUserID, cu.RegistrationDate) a
GROUP BY a.TOTOID, a.Age, a.EndDate, a.RegDate
ORDER BY TOTOID
"""
query = 'SET NOCOUNT ON; \n' + query
Users = pd.read_sql(query, cnx, index_col='TOTOID')
cnx.close()
Users

           DepositCount  DepositAmount  Age     RegDate     EndDate
TOTOID                                                             
100381794            44        22100.0   32  2015-10-18  2016-04-18
100381996            65       104900.0   31  2016-01-26  2016-07-26
100384513           169       235600.0   50  2016-06-19  2016-12-19
100384976            26        58000.0   36  2016-02-15  2016-08-15
100386754             3         2400.0   30  2016-12-01  2017-06-01
...                 ...            ...  ...         ...         ...
101232101            32        39900.0   30  2022-08-24  2023-02-24
101236274            79      1896400.0   26  2022-09-22  2023-03-22
101242982             1         1000.0   39  2022-11-02  2023-05-02
101243717             5         4000.0   51  2022-11-07  2023-05-07
101244367             4        51500.0   33  2022-11-10  2023-05-10

[174 rows x 5 columns]


In [21]:
Users.index.value_counts()

100381794    1
101075827    1
101065240    1
101065542    1
101066702    1
            ..
100737247    1
100745361    1
100748759    1
100756080    1
101244367    1
Name: TOTOID, Length: 174, dtype: int64

## Bets

In [11]:
cnx: pyodbc.Connection = pyodbc.connect(
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=DWH;"
    "Database=dwOper;"
    "Trusted_Connection=yes;")
query = f"""
    DROP TABLE IF EXISTS #cg
    SELECT cg.GameID,
        cg.GameCategoryID,
        cg.GameProviderID,
        cg.Name_en
    INTO #cg
    FROM C_Game cg

    DROP TABLE IF EXISTS #gc
    SELECT gc.GameCategoryID,
        gc.GameCategoryName
    INTO #gc
    FROM C_GameCategory gc

    DROP TABLE IF EXISTS #gp
    SELECT gp.GameProviderID,
        gp.GameProviderName
    INTO #gp
    FROM C_GameProvider gp

    DROP TABLE IF EXISTS #a
    SELECT #cg.GameID,
        #cg.Name_en,
        #gc.GameCategoryName,
        #gp.GameProviderName
    INTO #a
    FROM #cg
    LEFT JOIN #gc ON #cg.GameCategoryID = #gc.GameCategoryID
    LEFT JOIN #gp ON #cg.GameProviderID = #gp.GameProviderID

    DROP TABLE IF EXISTS #cas
    SELECT u.PartnerUserID,
        count(*) as BetCount,
        SUM(a.BetAmount),
        g.GameID
    INTO #cas
    FROM (
    SELECT o.UserID,
        o.GameID,
        (IIF(o.CalculationDate < '2021-03-01', o.OrderAmount,
        CASE WHEN cg.GameProviderID IN (48, 10) AND o.TypeId IN (1, 5, 8, 18, 33) THEN o.OrderAmount
        WHEN cg.GameProviderID NOT IN (48, 10) THEN o.OrderAmount ELSE 0 END)) AS BetAmount
    FROM casino.orders o
    INNER JOIN C_Game cg ON cg.GameID = o.GameID
    INNER JOIN VIEW_PlatformPartnerUsers_TotogamingAm u ON o.UserID = u.UserID
    INNER JOIN C_GameProvider gp ON gp.GameProviderID = cg.GameProviderID
    WHERE o.CalculationDate <= CAST(DATEADD(MONTH, 6, CAST(u.RegistrationDate AS DATE)) AS Date)
    AND u.PartnerUserId IN {tuple(Users.index)}
    AND gp.GameProviderID <> 3
    AND OrderStateID IN (2, 3, 5, 6)
    AND o.OperationTypeID IN (3, 299)
    AND IIF(cg.GameProviderID IN (48, 10)
    AND o.CalculationDate < '2021-03-01', o.TypeId, 0) IN (0, 1, 5, 8, 18, 33)
    ) a
    INNER JOIN C_Game g ON g.GameID = a.GameID
    INNER JOIN VIEW_PlatformPartnerUsers_TotogamingAm u ON u.UserID = a.UserID
    WHERE u.isDeleted = 0
    AND u.UserTypeID NOT IN (1, 20, 3, 21)
    AND u.UserName NOT LIKE 'test%'
    AND u.UserName NOT LIKE '%TestClient%'
    GROUP BY u.PartnerUserID,
        g.GameID

    SELECT * FROM (SELECT #cas.PartnerUserId AS TOTOID,
        SUM(#cas.BetCount) AS Bet_count,
        SUM(#cas.BetAmount) AS Bet_amount,
        #b.Type AS Category
    FROM #cas
    LEFT JOIN (SELECT *, (
    CASE WHEN #a.Name_en LIKE '%HabaneroJackpot%'
        THEN 'Other'
    WHEN #a.GameProviderName LIKE '%Kiron%'
        THEN 'Virtual Games'
    WHEN #a.Name_en LIKE '%Greyhound Racing%'
        THEN 'Virtual Games'
    WHEN #a.Name_en LIKE '%Penalty Shootout%'
        THEN 'Virtual Games'
    WHEN #a.Name_en LIKE '%Flat Horse Racing%'
        THEN 'Virtual Games'
    WHEN #a.Name_en LIKE '%Fantastic League%'
        THEN 'Virtual Games'
    WHEN #a.Name_en LIKE '%Horse Racing%'
        THEN 'Virtual Games'
    WHEN #a.Name_en LIKE '%English league%'
        THEN 'Virtual Games'
    WHEN #a.GameProviderName LIKE '%Digitain%' AND #a.Name_en LIKE '%Football Single Match%'
        THEN 'Virtual Games'
    WHEN #a.GameProviderName LIKE '%GlobalBet%'
        THEN 'Virtual Games'
    WHEN #a.GameProviderName LIKE '%GoldenRace%' AND #a.Name_en in ('Keno','Keno Deluxe')
        THEN 'TVGames'
    WHEN #a.GameProviderName LIKE '%Golden%'
        THEN 'Virtual Games'
    WHEN #a.GameProviderName LIKE '%EventBet%'
        THEN 'Poker'
    WHEN #a.GameProviderName LIKE '%Betgames%'
        THEN 'TVGames'
    WHEN #a.GameProviderName LIKE '%Atmosfera%' AND #a.Name_en in ('Bingo 37','Keno','Bingo 38')
        THEN 'TVGames'
    WHEN #a.GameProviderName = 'FLG' AND #a.Name_en in ('Keno Gold')
        THEN 'TVGames'
    WHEN #a.GameProviderName LIKE '%Betongames%'
        THEN 'Betongames'
    WHEN #a.GameProviderName LIKE '%Evolution%'
        THEN 'Live Casino'
    WHEN #a.GameProviderName LIKE '%LiveCasino%'
        THEN 'Live Casino'
    WHEN #a.GameProviderName LIKE '%EzugiOriginal%'
        THEN 'Live Casino'
    WHEN #a.GameProviderName LIKE '%LuckyStreak%'
        THEN 'Live Casino'
    WHEN #a.GameProviderName LIKE '%VivoGaming%'
        THEN 'Live Casino'
    WHEN #a.GameProviderName LIKE '%NetEntLiveCasino%'
        THEN 'Live Casino'
    WHEN #a.GameProviderName LIKE '%TV Bet%'
        THEN 'TVGames'
    WHEN #a.GameProviderName LIKE '%BetGames%'
        THEN 'TVGames'
    WHEN #a.GameCategoryName LIKE '%P2P%'
        THEN 'P2P'
    WHEN #a.GameProviderName LIKE '%GGPoker%'
        THEN 'Poker'
    WHEN #a.GameCategoryName LIKE '%Slot%'
        THEN 'Slots'
    WHEN #a.GameProviderName LIKE '%InBet%'
        THEN 'Virtual Games'
    WHEN #a.GameProviderName LIKE '%Leap%'
        THEN 'Virtual Games'
    WHEN #a.GameProviderName LIKE '%Digitain Live Casino%'
        THEN 'Live Casino'
    WHEN #a.GameProviderName LIKE '%Digitain%'
        THEN 'Sport'
    WHEN #a.GameProviderName LIKE '%Pragmatic%' AND #a.GameCategoryName LIKE '%Baccarat%'
        THEN 'Live Casino'
    WHEN #a.GameProviderName LIKE '%Pragmatic%' AND #a.GameCategoryName LIKE '%Blackjack%'
        THEN 'Live Casino'
    WHEN #a.GameProviderName LIKE '%Pragmatic%' AND #a.GameCategoryName LIKE '%Roulette%'
        THEN 'Live Casino'
    WHEN #a.GameProviderName LIKE '%Pragmatic%' AND #a.GameCategoryName LIKE '%TV Show%'
        THEN 'Live Casino'
    WHEN #a.GameProviderName LIKE '%Pragmatic%' AND #a.GameCategoryName LIKE '%GameShow%'
        THEN 'Live Casino'
    WHEN #a.GameProviderName LIKE '%Pragmatic%' AND #a.Name_en LIKE '%Mega Sic Bo%'
        THEN 'Live Casino'
    WHEN #a.GameProviderName LIKE '%Pragmatic%' AND #a.Name_en LIKE '%Dragon Tiger%'
        THEN 'Live Casino'
    WHEN #a.GameProviderName LIKE '%Pragmatic%' AND #a.Name_en LIKE '%Andar Bahar%'
        THEN 'Live Casino'
    WHEN #a.Name_en LIKE '%Dynamic Roulette 120x%'
        THEN 'Live Casino'
    WHEN #a.Name_en LIKE '%Live European Roulette%'
        THEN 'Live Casino'
    WHEN #a.GameProviderName LIKE '%Atmosfera%' AND #a.Name_en LIKE '%Live Roulette%'
        THEN 'Live Casino'
    WHEN #a.GameProviderName LIKE '%Atmosfera%' AND #a.Name_en LIKE '%Music Wheel%'
        THEN 'Live Casino'
    WHEN #a.GameProviderName LIKE '%Atmosfera%' AND #a.Name_en LIKE '%Auto Roulette%'
        THEN 'Live Casino'
    WHEN #a.GameProviderName LIKE '%XPG%'
        THEN 'Live Casino'
    WHEN #a.GameProviderName LIKE '%EGT%' AND #a.Name_en LIKE '%Onyx Roulette%'
        THEN 'Live Casino'
    WHEN #a.GameProviderName LIKE '%EGT%' AND #a.Name_en LIKE '%Live Speed Roulette%'
        THEN 'Live Casino'
    WHEN #a.GameProviderName LIKE '%EGT%' AND #a.Name_en LIKE '%Vegas Roulette 500x%'
        THEN 'Live Casino'
    WHEN #a.GameProviderName LIKE '%PlayngoArmenia%' AND #a.Name_en LIKE '%European Roulette Pro%'
        THEN 'Slots'
    WHEN #a.GameProviderName LIKE '%EGT%' AND #a.Name_en LIKE '%Burning Keno Plus%'
        THEN 'Slots'
    WHEN #a.GameProviderName LIKE '%1x2%' AND #a.Name_en LIKE '%Instant Football%'
        THEN 'Virtual Games'
    WHEN #a.GameProviderName LIKE '%EGT%' AND #a.Name_en LIKE '%European Roulette%'
        THEN 'Slots'
    ELSE 'Other'
    END) Type
    FROM #a ) #b ON #cas.GameID = #b.GameID
    GROUP BY #cas.PartnerUserId, #b.Type

    UNION ALL

    SELECT u.PartnerUserID AS TOTOID,
        COUNT(o.OrderID) AS Order_count,
        SUM(o.OrderAmount) AS Order_amount,
        'Sports' AS Category
    FROM VIEW_sport_PartnerUser_TotogamingAm u
    INNER JOIN VIEW_sport_Orders_TotogamingAm o ON o.UserID = u.UserID
    WHERE o.OrderDate_DT <= CAST(DATEADD(MONTH, 6, CAST(u.RegistrationDate AS DATE)) AS Date)
   -- AND u.PartnerID IN {tuple(Users.index)}
    AND o.OrderStateID IN (2, 3, 5, 6)
    AND u.UserTypeID NOT IN (1, 20, 3, 21)
    AND u.UserName NOT LIKE 'test%'
    AND u.UserName NOT LIKE '%TestClient%'
    GROUP BY u.PartnerUserId) AS bc
    ORDER BY bc.TOTOID
    """
query = 'SET NOCOUNT ON; \n' + query
bets_games = pd.read_sql(query, cnx, index_col='TOTOID')
cnx.close()
print(bets_games)

           Bet_count  Bet_amount    Category
TOTOID                                      
100723056          1         0.0       Other
100723056       5853     13520.0       Slots
100730106        261       270.0       Slots
100730106          3         0.0         P2P
100738271        796        95.0       Slots
...              ...         ...         ...
101238891        424        80.0       Slots
101242095         14         0.0  Betongames
101242095        142      6985.0       Slots
101242180       1904       126.0       Slots
101242180         40       210.0  Betongames

[250 rows x 3 columns]
           Bet_count  Bet_amount    Category
TOTOID                                      
100722851         25        95.0       Slots
100726183        341     11140.0       Other
100726183        192        60.0       Slots
100730314        508        45.0       Slots
100730395        380       670.0  Betongames
...              ...         ...         ...
101242982         50        30.

In [22]:
bets_games.index.value_counts()

101148349    5
101146320    5
101220080    4
101227570    4
101197088    4
            ..
101013236    1
100748759    1
101018428    1
100730314    1
100722851    1
Name: TOTOID, Length: 108, dtype: int64

## Bets History

In [12]:
cnx: pyodbc.Connection = pyodbc.connect(
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=DWH;"
    "Database=dwOper;"
    "Trusted_Connection=yes;")
query = f"""
DROP TABLE IF EXISTS #cas
SELECT u.PartnerUserID,
       count(*) as BetCount,
       SUM(a.BetAmount),
       a.Date
INTO #cas
FROM (SELECT o.UserID,
             o.GameID,
             (IIF(o.CalculationDate < '2021-03-01', o.OrderAmount,
                  CASE
                      WHEN cg.GameProviderID IN (48, 10) AND o.TypeId IN (1, 5, 8, 18, 33) THEN o.OrderAmount
                      WHEN cg.GameProviderID NOT IN (48, 10) THEN o.OrderAmount
                      ELSE 0 END))           AS BetAmount,
             o.CalculationDate_DT AS Date
      FROM casino.orders o
               INNER JOIN C_Game cg ON cg.GameID = o.GameID
               INNER JOIN VIEW_PlatformPartnerUsers_TotogamingAm u ON o.UserID = u.UserID
               INNER JOIN C_GameProvider gp ON gp.GameProviderID = cg.GameProviderID
      WHERE o.CalculationDate <= CAST(DATEADD(MONTH, 6, CAST(u.RegistrationDate AS DATE)) AS Date)
        AND gp.GameProviderID <> 3
        AND u.PartnerUserId IN {tuple(Users.index)}
        AND OrderStateID IN (2, 3, 5, 6)
        AND o.OperationTypeID IN (3, 299)
        AND IIF(cg.GameProviderID IN (48, 10)
                    AND o.CalculationDate < '2021-03-01', o.TypeId, 0) IN (0, 1, 5, 8, 18, 33)) a
INNER JOIN VIEW_PlatformPartnerUsers_TotogamingAm u ON u.UserID = a.UserID
WHERE u.isDeleted = 0
  AND u.UserTypeID NOT IN (1, 20, 3, 21)
  AND u.UserName NOT LIKE 'test%'
  AND u.UserName NOT LIKE '%TestClient%'
GROUP BY u.PartnerUserID,
         a.Date

SELECT * FROM (
SELECT #cas.PartnerUserId  AS TOTOID,
       #cas.Date           AS Date,
       SUM(#cas.BetCount)  AS Bet_count,
       SUM(#cas.BetAmount) AS Bet_amount
FROM #cas
GROUP BY #cas.PartnerUserId, #cas.Date

UNION ALL

SELECT u.PartnerUserID               AS TOTOID,
       o.OrderDate_DT                AS DATE ,
       COUNT(o.OrderID)              AS Bet_count,
       SUM(o.OrderAmount)            AS Bet_amount
FROM VIEW_sport_PartnerUser_TotogamingAm u
         INNER JOIN VIEW_sport_Orders_TotogamingAm o ON o.UserID = u.UserID
WHERE o.OrderDate_DT <= CAST(DATEADD(MONTH, 6, CAST(u.RegistrationDate AS DATE)) AS Date)
  AND o.OrderStateID IN (2, 3, 5, 6)
  --AND u.PartnerUserId IN {tuple(Users.index)}
  AND u.UserTypeID NOT IN (1, 20, 3, 21)
  AND u.UserName NOT LIKE 'test%'
  AND u.UserName NOT LIKE '%TestClient%'
GROUP BY u.PartnerUserId, o.OrderDate_DT) AS bc
ORDER BY bc.TOTOID
"""
query = 'SET NOCOUNT ON; \n' + query
bets = pd.read_sql(query, cnx, index_col='TOTOID')
cnx.close()
print(bets)

                 Date  Bet_count  Bet_amount
TOTOID                                      
100723056  2020-05-06       1756      6000.0
100723056  2020-05-20       1935      3000.0
100723056  2020-05-05       1730     10800.0
100723056  2020-05-03        383      2700.0
100723056  2020-05-21          4       300.0
...               ...        ...         ...
101242180  2022-11-12         22        10.0
101242180  2022-10-31         27        11.0
101242180  2022-11-04          4        10.0
101242180  2022-11-05          8        30.0
101242180  2022-11-03         54        30.0

[7286 rows x 3 columns]
                 Date  Bet_count  Bet_amount
TOTOID                                      
100722851  2020-05-18         25        95.0
100726183  2020-05-06        258      6920.0
100726183  2020-06-04         83      2520.0
100726183  2020-06-05        192        60.0
100730314  2020-05-16        101        30.0
...               ...        ...         ...
101243717  2022-11-07       10

## Deposit History

In [13]:
cnx: pyodbc.Connection = pyodbc.connect(
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=DWH;"
    "Database=dwOper;"
    "Trusted_Connection=yes;")
query = f"""
SELECT a.TOTOID,
    SUM(a.DepositCount) AS DepositCount,
    SUM(a.DepositAmount) AS DepositAmount,
    a.DepositDate AS Date
FROM
(SELECT cu.PartnerUserID AS TOTOID,
    COUNT(p.PaymentID) DepositCount,
    SUM(p.Amount) DepositAmount,
    CAST(p.modify_date AS DATE) AS DepositDate
FROM Payment p
INNER JOIN VIEW_PlatformPartnerUsers_TotogamingAm cu ON p.UserID = cu.UserID
LEFT JOIN C_PaymentSystem sp ON sp.PaymentSystemId = p.PaymentSystemID
WHERE CAST(p.modify_date AS DATE) <= CAST(DATEADD(MONTH, 6, CAST(cu.RegistrationDate AS DATE)) AS DATE)
    -- AND cu.PartnerUserID IN {tuple(Users.index)}
	AND p.PaymentStatusID = 8
    AND p.PaymentTypeID = 2
    AND p.SourceID = 2 --Casino Source
    AND sp.PaymentSystemName NOT IN ('PokerTransfer','TRANSFER')
GROUP BY cu.PartnerUserID, CAST(p.modify_date AS Date)

UNION

SELECT cu.PartnerUserID AS TOTOID,
    COUNT(p.PaymentID) DepositCount,
    SUM(p.Amount) DepositAmount,
    CAST(p.modify_date AS Date) AS DepositDate
FROM Payment p
INNER JOIN VIEW_sport_PartnerUser_TotogamingAm cu ON p.UserID = cu.UserID
LEFT JOIN C_PaymentSystem sp ON sp.PaymentSystemId = p.PaymentSystemID
WHERE CAST(p.modify_date AS DATE) <= CAST(DATEADD(MONTH, 6, CAST(cu.RegistrationDate AS DATE)) AS DATE)
	-- AND cu.PartnerUserID IN {tuple(Users.index)}
    AND p.TransactionTypeID IN (62,65,70,73,87,88,102,103,75,71)
    AND p.Fund_StatusID IN (3,23,31,53,63,106)
    AND p.SourceID = 1  --Sport Source
    AND sp.PaymentSystemName NOT IN ('Casino Transfer','BONUS')
GROUP BY cu.PartnerUserID, CAST(p.modify_date AS Date)) a
GROUP BY a.TOTOID, a.DepositDate
ORDER BY TOTOID
"""
query = 'SET NOCOUNT ON; \n' + query
deposit_hist = pd.read_sql(query, cnx, index_col='TOTOID')
cnx.close()

In [20]:
print(Users)
print(bets)
print(bets_games)
print(deposit_hist)

           DepositCount  DepositAmount  Age     RegDate     EndDate
TOTOID                                                             
100381794            44        22100.0   32  2015-10-18  2016-04-18
100381996            65       104900.0   31  2016-01-26  2016-07-26
100384513           169       235600.0   50  2016-06-19  2016-12-19
100384976            26        58000.0   36  2016-02-15  2016-08-15
100386754             3         2400.0   30  2016-12-01  2017-06-01
...                 ...            ...  ...         ...         ...
101232101            32        39900.0   30  2022-08-24  2023-02-24
101236274            79      1896400.0   26  2022-09-22  2023-03-22
101242982             1         1000.0   39  2022-11-02  2023-05-02
101243717             5         4000.0   51  2022-11-07  2023-05-07
101244367             4        51500.0   33  2022-11-10  2023-05-10

[174 rows x 5 columns]
                 Date  Bet_count  Bet_amount
TOTOID                                      
10

In [17]:
# for i in range(len(bets_for_user)-1):
#     date_1 = bets_for_user.iloc[i, 0]
#     date_2 = bets_for_user.iloc[i+1, 0]
#     # d1 = datetime.strptime(date_1, "%Y-%m-%d")
#     # d2 = datetime.strptime(date_2, "%Y-%m-%d")
#     delta = date_2-date_1
#     print(delta.days)